In [29]:
import struct
import socket

# Dynamic Host Configuration Protocol

The Dynamic Host Configuration Protocol (DHCP) is a network management protocol used on UDP/IP networks whereby a DHCP server dynamically assigns an IP address and other network configuration parameters to each device on a network so they can communicate with other IP networks.A DHCP server enables computers to request IP addresses and networking parameters automatically from the Internet service provider (ISP), reducing the need for a network administrator or a user to manually assign IP addresses to all network devices.In the absence of a DHCP server, a computer or other device on the network needs to be manually assigned an IP address, or to assign itself an APIPA address, which will not enable it to communicate outside its local subnet.

DHCP can be implemented on networks ranging in size from home networks to large campus networks and regional Internet service provider networks.A router or a residential gateway can be enabled to act as a DHCP server. Most residential network routers receive a globally unique IP address within the ISP network. Within a local network, a DHCP server assigns a local IP address to each device connected to the network. 

## DHCP Operation

The DHCP employs a connectionless service model, using the User Datagram Protocol (UDP). It is implemented with two UDP port numbers for its operations. *UDP port number 67 is the destination port of a server, and UDP port number 68 is used by the client.*

DHCP operations fall into four phases: 
1. server discovery, 
2. IP lease offer, 
3. IP lease request, and 
4. IP lease acknowledgement. 

These stages are often abbreviated as **DORA** for discovery, offer, request, and acknowledgement.

<img src='images/DHCP_session.png' width=30% >


The DHCP operation begins with clients broadcasting a request. If the client and server are on different subnets, a DHCP Helper or DHCP Relay Agent may be used. Clients requesting renewal of an existing lease may communicate directly via UDP unicast, since the client already has an established IP address at that point. Additionally, there is a BROADCAST (B) flag the client can use to indicate in which way (broadcast or unicast) it can receive the DHCPOFFER: 0x8000 for broadcast, 0x0000 for unicast.[5] Usually, the DHCPOFFER is sent through unicast. For those hosts which cannot accept unicast packets before IP addresses are configured, this flag can be used to work around this issue. 

## DHCP Messag format

The following figure shows the DHCP message format:
<img src='images/dhcp_format.png' width=30%>

This format includes the following fields: http://www.tcpipguide.com/free/t_DHCPMessageFormat.htm#Table_189


** Exercise: ** Implement `dhcp_message()` which generates the DHCP packet. Use `struct.pack` (https://docs.python.org/3/library/struct.html) and proper format characters to add different fields to the packet.  


In [30]:
def dhcp_message(operation_code,
                 hardware_type,
                 hardware_address_length,
                 transaction_identifier,
                 client_IP_address,
                 your_IP_address,
                 server_IP_address,
                 gateway_IP_address,
                 client_hardware_address
                 ):
    # make empty binary string
    dhcp_message = b''
    
    # Field name: Operation Code (OP) 
    # Size: 1 Byte
    OP = struct.pack('B',operation_code)
    dhcp_message += OP
    
 
    # Field name: Hardware Type: HType
    # Size: 1 Byte
    HType = struct.pack('B',hardware_type)
    dhcp_message+= HType
    
    
    # Field name: Hardware Address Length: HLen
    # Size: 1 Byte
    # Value: 6
    HLen = struct.pack('B',hardware_address_length)
    dhcp_message+= HLen
    
    
    # Field name: Hops
    # Size: 1 Byte
    # Value: 0 (no relay nodes)
    Hops = struct.pack('B',0)
    dhcp_message += Hops
    
     
    
    # Field name: Transaction Identifier (XID) 
    # Size: 4 Byte(s)
    # Value: A 32-bit identification field generated by the client,
    # to allow it to match up the request with replies received from DHCP servers.
    XID = struct.pack('I',transaction_identifier)
    dhcp_message += XID
    
    
    # Field name: Seconds (Secs)
    # Size: 2 Byte(s)
    # Value: For DHCP, it is defined as the number of seconds elapsed 
    # since a client began an attempt to acquire or renew a lease
    # Value: 0
    Secs = struct.pack('H',0)
    dhcp_message += Secs
    
    # Field name: Flags (Flags)
    # Size: 2 Byte(s)
    # Value: 0
    Flags = struct.pack('H',0)
    dhcp_message += Flags
    
    # Field name: Client IP Address (CIAddr)
    # Size: 4 Byte(s)
    # Hint: use  socket.inet_aton(ip_string)
    CIAddr = socket.inet_aton(client_IP_address)
    dhcp_message+= CIAddr
    
    # Field name: Your IP Address (YIAddr)
    # Size: 4 Byte(s)
    # Hint: use  socket.inet_aton(ip_string)
    YIAddr = socket.inet_aton(your_IP_address)
    dhcp_message+= YIAddr
    
    
    # Field name: Server IP Address (SIAddr)
    # Size: 4 Byte(s)
    # Hint: use  socket.inet_aton(ip_string)
    SIAddr = socket.inet_aton(server_IP_address)
    dhcp_message+= SIAddr
    
    # Field name: Gateway IP Address (GIAddr)
    # Size: 4 Byte(s)
    # Hint: use  socket.inet_aton(ip_string)
    GIAddr = socket.inet_aton(gateway_IP_address)
    dhcp_message+= GIAddr
    
     # Field name: Client Hardware Address: (CHAddr)
    # Size: 16 Byte(s)
    # the first 6 bytes are for the MAC address
    # Create empty binary string
    CHAddr = b''
    for mac_field in client_hardware_address.split(':'):
        # convert Hex to int and pack it as one byte
        CHAddr += struct.pack('B', int(mac_field,16))
    
    # the rest of the bytes will be set to zero
    for i in range(10):
        CHAddr += struct.pack('B',0)
    
    dhcp_message+= CHAddr
    
    
    return dhcp_message

In [31]:
dhcp_msg = dhcp_message(operation_code = 1,
                            hardware_type = 1,
                            hardware_address_length = 6,
                            transaction_identifier=1206,
                            client_IP_address='0.0.0.0',
                            your_IP_address='0.0.0.0',
                            server_IP_address='0.0.0.0',
                            gateway_IP_address='0.0.0.0',
                            client_hardware_address='fe:1d:20:1c:f6:8d'
                           )
client_hardware_address='fe:1d:20:1c:f6:8d'
print('dhcp message length: {0} bytes'.format(len(dhcp_msg)))
_mac = []
for i in range(-16,-10):
    _mac.append(hex(struct.unpack('B',dhcp_msg[i:i+1])[0]))
_mac  = ':'.join(_mac).replace('0x','')
print('HW MAC check (1): ', client_hardware_address == _mac)


dhcp message length: 44 bytes
HW MAC check (1):  True


** Expected output: ** 

- dhcp message length: 44 bytes
- HW MAC check (1):  True


## DHCP Discovery
The DHCP client broadcasts a DHCPDISCOVER message on the network subnet using the **destination address 255.255.255.255 or the specific subnet broadcast address**. A DHCP client may also request its last known IP address. If the client remains connected to the same network, the server may grant the request. Otherwise, it depends whether the server is set up as authoritative or not. An authoritative server denies the request, causing the client to issue a new request. A non-authoritative server simply ignores the request, leading to an implementation-dependent timeout for the client to expire the request and ask for a new IP address.

** Exercise: ** Generate a DHCPDISCOVER message using `dhcp_message(:)` implemented above. Assume that this DHCP discovery message is sent over Ethernet LAN. 



In [32]:
operation_code = 1
hardware_type = 1
hardware_address_length = 6
transaction_identifier = 1200
client_IP_address = '0.0.0.0'
your_IP_address = '0.0.0.0'
server_IP_address = '0.0.0.0'
gateway_IP_address = '0.0.0.0'
client_hardware_address='fe:1d:20:1c:f6:8d'
DHCPDISCOVER = dhcp_message(operation_code = operation_code,
                            hardware_type = hardware_type,
                            hardware_address_length = hardware_address_length,
                            transaction_identifier=1206,
                            client_IP_address=client_IP_address,
                            your_IP_address=your_IP_address,
                            server_IP_address=server_IP_address,
                            gateway_IP_address=gateway_IP_address,
                            client_hardware_address=client_hardware_address
                           )


test_HLen = struct.unpack('B', DHCPDISCOVER[1:2])[0]
print('Sanity Check: ', test_HLen == hardware_type == 1)


Sanity Check:  True


** Expected output: ** Sanity Check:  True

### DHCP offer

When a DHCP server receives a DHCPDISCOVER message from a client, which is an IP address lease request, the DHCP server reserves an IP address for the client and makes a lease offer by sending a DHCPOFFER message to the client. This message contains the client's client id (traditionally a MAC address), the IP address that the server is offering, the subnet mask, the lease duration, and the IP address of the DHCP server making the offer. 


** Exercise: ** Generate DHCPOFFER message using `dhcp_message(:)`. In this scenario, the server at 192.168.20.1, offers the client an IP address of 192.168.20.20 

In [33]:
operation_code = 1
hardware_type = 1
hardware_address_length = 6
transaction_identifier = 1200
client_IP_address = '0.0.0.0'
your_IP_address = '192.168.20.20'
server_IP_address = '192.168.20.1'
gateway_IP_address = '0.0.0.0'
client_hardware_address='fe:1d:20:1c:f6:8d'
DHCPOFFER = dhcp_message(operation_code = operation_code,
                            hardware_type = hardware_type,
                            hardware_address_length = hardware_address_length,
                            transaction_identifier=1206,
                            client_IP_address=client_IP_address,
                            your_IP_address=your_IP_address,
                            server_IP_address=server_IP_address,
                            gateway_IP_address=gateway_IP_address,
                            client_hardware_address=client_hardware_address
                           )

test = socket.inet_ntoa(DHCPOFFER[12:16])
print('Sanity Check (1): ', test == '0.0.0.0')
test = socket.inet_ntoa(DHCPOFFER[16:20])
print('Sanity Check (2): ', test == '192.168.20.20')
test = socket.inet_ntoa(DHCPOFFER[20:24])
print('Sanity Check (3): ', test == '192.168.20.1')

Sanity Check (1):  True
Sanity Check (2):  True
Sanity Check (3):  True


** Expected output: ** 

- Sanity Check (1):  True
- Sanity Check (2):  True
- Sanity Check (3):  True



### DHCP request
In response to the DHCP offer, the client replies with a DHCPREQUEST message, broadcast to the server,[a] requesting the offered address. A client can receive DHCP offers from multiple servers, but it will accept only one DHCP offer. Based on required server identification option in the request and broadcast messaging, servers are informed whose offer the client has accepted.

** Exercise: ** Generate DHCPREQUEST message using `dhcp_message(:)`. Consider this is the DHCPREQUEST in response to the DHCPOFFER above. 


In [34]:
operation_code = 1
hardware_type = 1
hardware_address_length = 6
transaction_identifier = 1200
client_IP_address = '0.0.0.0'
your_IP_address = '0.0.0.0'
server_IP_address = '192.168.20.1'
gateway_IP_address = '0.0.0.0'
client_hardware_address='fe:1d:20:1c:f6:8d'
DHCPREQUEST = dhcp_message(operation_code = operation_code,
                            hardware_type = hardware_type,
                            hardware_address_length = hardware_address_length,
                            transaction_identifier=1206,
                            client_IP_address=client_IP_address,
                            your_IP_address=your_IP_address,
                            server_IP_address=server_IP_address,
                            gateway_IP_address=gateway_IP_address,
                            client_hardware_address=client_hardware_address
                           )

test = socket.inet_ntoa(DHCPREQUEST[12:16])
print('Sanity Check (1): ', test == '0.0.0.0')
test = socket.inet_ntoa(DHCPREQUEST[16:20])
print('Sanity Check (2): ', test == '0.0.0.0')
test = socket.inet_ntoa(DHCPREQUEST[20:24])
print('Sanity Check (3): ', test == '192.168.20.1')

Sanity Check (1):  True
Sanity Check (2):  True
Sanity Check (3):  True


** Expected output: ** 

- Sanity Check (1):  True
- Sanity Check (2):  True
- Sanity Check (3):  True


### DHCP acknowledgement

When the DHCP server receives the DHCPREQUEST message from the client, the configuration process enters its final phase. The acknowledgement phase involves sending a DHCPACK packet to the client. This packet includes the lease duration and any other configuration information that the client might have requested. At this point, the IP configuration process is completed. *The protocol expects the DHCP client to configure its network interface with the negotiated parameters.*

** Exercise: ** Generate DHCACK message using `dhcp_message(:)`. Consider this is the DHCACK
in response to the DHCPREQUEST above. 


In [35]:
operation_code = 1
hardware_type = 1
hardware_address_length = 6
transaction_identifier = 1200
client_IP_address = '0.0.0.0'
your_IP_address = '192.168.20.20'
server_IP_address = '192.168.20.1'
gateway_IP_address = '0.0.0.0'
client_hardware_address='fe:1d:20:1c:f6:8d'
DHCACK = dhcp_message(operation_code = operation_code,
                            hardware_type = hardware_type,
                            hardware_address_length = hardware_address_length,
                            transaction_identifier=1206,
                            client_IP_address=client_IP_address,
                            your_IP_address=your_IP_address,
                            server_IP_address=server_IP_address,
                            gateway_IP_address=gateway_IP_address,
                            client_hardware_address=client_hardware_address
                           )

test = socket.inet_ntoa(DHCACK[12:16])
print('Sanity Check (1): ', test == '0.0.0.0')
test = socket.inet_ntoa(DHCACK[16:20])
print('Sanity Check (2): ', test == '192.168.20.20')
test = socket.inet_ntoa(DHCACK[20:24])
print('Sanity Check (3): ', test == '192.168.20.1')

Sanity Check (1):  True
Sanity Check (2):  True
Sanity Check (3):  True
